In [60]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from random import shuffle

In [61]:
csv_files = [
    "data/pokemon_rb_gen1.csv",
    "data/pokemon_gs_gen2.csv",
    "data/pokemon_rs_gen3.csv",
    "data/pokemon_dp_gen4.csv",
    "data/pokemon_bw_gen5.csv",
    "data/pokemon_xy_gen6.csv",
    "data/pokemon_sm_gen7.csv",
    "data/pokemon_swsh_gen8.csv",
    "data/pokemon_sv_gen9.csv",
]

dataframes = []

for i, file in enumerate(csv_files):
    df = pd.read_csv(file)
    df['Generation'] = i + 1
    dataframes.append(df)

combined_data = pd.concat(dataframes, ignore_index=True)

combined_data.to_csv("all_pokemon_generation_data.csv", index=False)

Połączenie wszystkich generacji pokemonów.

In [62]:
data = pd.read_csv("all_pokemon_generation_data.csv")
data = data.drop_duplicates(subset='Name', keep='first')

In [63]:
data.head(5000)

,No,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd,Generation
0,10,Caterpie,bug,NaN,45,30,35,20,NaN,45,1
1,11,Metapod,bug,NaN,50,20,55,25,NaN,30,1
2,12,Butterfree,"bug, flying",NaN,60,45,50,80,NaN,70,1
3,13,Weedle,"bug, poison",NaN,40,35,30,20,NaN,50,1
4,14,Kakuna,"bug, poison",NaN,45,25,50,25,NaN,35,1
...,...,...,...,...,...,...,...,...,...,...,...
8310,960,Wiglett,water,"Gooey, Rattled, Sand Veil",10,55,25,35,25.0,95,9
8311,961,Wugtrio,water,"Gooey, Rattled, Sand Veil",35,100,50,50,70.0,120,9
8312,963,Finizen,water,Water Veil,70,45,40,45,40.0,75,9
8313,964,Palafin,water,Zero to Hero,100,70,72,53,62.0,100,9


Póżniejsze generacje pokemonów mają kolumne speed defense oraz nazwę specjalnych umiejętności. Myślę, że specjalne umiejętności nie będą potrzebne w tym przypadku ponieważ nie jestem w stanie stwierdzić jak działają i jaką mają moc w jaki sposób mogą wpłynąć na przewidywania wyników. Oczywiście, specjalna umiejętność znacząco wpływa na siłe pokemona, jednak nie posiadam danych dotyczących siły tej umiejętności.

In [64]:
X = data[['HP', 'Att', 'Def', 'S.Att', 'S.Def', 'Spd']].values
X = X / X.max(axis=0) # Normalizacja

Y = data[['HP', 'Att', 'Def', 'S.Att', 'S.Def', 'Spd']].sum(axis=1).values
Y = (Y > np.percentile(Y, 75)).astype(int)

Normalizacja i podział pokemonów zero jedynkowo na silne (1) i słabe (0).

In [65]:
data = data.drop(columns='Abilities')

Usunięcie kolumny abillites. Zdecydowałem się na usunięcie tej kolumny ponieważ należało by zgłębić każdą poszczególną umiejętność aby móc ocenić jej dodatkową siłe co pomogło by w lepszym przewidywaniu modelu. 

In [66]:
data['S.Def'] = data['S.Def'].fillna(data['S.Def'].mean())
data['S.Def_missing'] = data['S.Def'].isna().astype(int)

Wypełnienie brakujących wartości medianą wszytkich S.Def pokemonów oraz ustawienie flagi aby model mógł uczyć się, że dane zostały wypełnione.

In [67]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)


Przetasowanie danych.

In [68]:
def init_params():
    W1 = np.random.rand(10, 6) - 0.5 # zmiana na 6 cech wejściowych
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(2, 10) - 0.5 # wejście: 2 klasy (słaby, silny)
    b2 = np.random.rand(2, 1) - 0.5

    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLu(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLu(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2

    return W1, b1, W2, b2

Algorytm sieci neuronowych zawierający inicjalizacje parametrów, propagację w przód, one-hot encoding etykiet i propagację wstecz i aktualizacje wag.

In [69]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    m = X.shape[1] # Liczna przykładów
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

Trening modelu poprzez algorytm iteracji parametrów w celu zmiejszenia błedu poprawy przewidywań, uaktualniania parametrów oraz śledzenia postępów uczenia się oraz precyzji.

In [70]:
W1, b1, W2, b2 = gradient_descent(X.T, Y, alpha=0.1, iterations=10000)

Iteration:  0
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  10
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  20
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  30
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  40
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  50
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  60
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  70
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  80
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  90
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  100
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  110
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  120
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  130
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 1]
0.7558365758754864
Iteration:  140
[

In [71]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def strongest_pokemon(X, data, W1, b1, W2, b2):
    predictions = make_predictions(X.T, W1, b1, W2, b2)
    strongest_idx = np.argmax(predictions)
    print("Najsilniejszy Pokemon: ", data.iloc[strongest_idx]['Name'])
    print(data.iloc[strongest_idx])

In [72]:
strongest_pokemon(X, data, W1, b1, W2, b2)

Najsilniejszy Pokemon:  Heatran
No                       485
Name                 Heatran
Type             fire, steel
HP                        91
Att                       90
Def                      106
S.Att                    130
S.Def                  106.0
Spd                       77
Generation                 4
S.Def_missing              0
Name: 0, dtype: object


In [73]:
data.head(1)

,No,Name,Type,HP,Att,Def,S.Att,S.Def,Spd,Generation,S.Def_missing
0,485,Heatran,"fire, steel",91,90,106,130,106.0,77,4,0


In [74]:
data.loc[data['Name']=='Arceus']

,No,Name,Type,HP,Att,Def,S.Att,S.Def,Spd,Generation,S.Def_missing
135,493,Arceus,bug,120,120,120,120,120.0,120,4,0


Model pokazuje błędnie. Model pokazuje po prostu pierwszy wylosowany rekord. Najsilniejszym pokemonem w całym uniwersum jest Arceus, zresztą nawet jego suma statystyk mówi sama za siebie. 